# Оцениваем регрессии на данных HS0 в разных спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные
- Фиксированные эффекты ~ ковид и ограничения
- Фиксированные эффекты ~ ковид и ограничения | инструментальные переменные
- В перспективе: суммарный экспорт/импорт в зависимости от ковида (+ IV)

In [34]:
library(arrow)
library(ivreg)
library(stargazer)
library(tidyverse)
library(marginaleffects)
# library(plm)
# library("lmtest")
library("sandwich")
library(fixest)

Warning message:
"пакет 'marginaleffects' был собран под R версии 4.4.2"


## Сырые данные

In [58]:
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
df$deaths_j = df$`deaths:r_j`
df$median_age_j = df$`median_age:r_j`
df$population_density_j = df$`population_density:r_j`
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,country:household_size:r_j,avg_hh_size:r_j,share_1_members:r_j,share_2_3_members:r_j,share_4_5_members:r_j,year:r_j,deaths_j,median_age_j,population_density_j,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,NA,NA,NA,NA,NA,2021,1.60279452,26.235,3.950483,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,SEN,8.66,6.92,11.47,15.48,2021,0.23334247,18.160,84.937007,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,KOR,2.34,31.75,48.10,19.27,2020,0.04320442,42.184,530.377275,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,CRI,3.21,12.07,48.80,31.61,2020,1.14320442,32.014,97.915257,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,BIH,3.50,11.50,37.07,41.57,2021,8.00158904,43.292,65.342441,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,MLT,2.67,22.64,49.28,25.60,2022,1.76879452,38.893,1575.193750,1.263014


In [45]:
m1 = feols(log(v/v19) ~ log(deaths) + log(deaths_j) +
 C8EV_International_travel_controls + C8EV_International_travel_controls_j| i + j + year,
 data = df)
m1 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
log(deaths)                          -0.015588   0.006061 -2.571959 0.010948
log(deaths_j)                        -0.003195   0.005269 -0.606428 0.545021
C8EV_International_travel_controls   -0.021174   0.013888 -1.524631 0.129167
C8EV_International_travel_controls_j -0.021157   0.013751 -1.538635 0.125710
                                        
log(deaths)                          *  
log(deaths_j)                           
C8EV_International_travel_controls      
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.981558     Adj. R2: 0.038294
                 Within R2: 2.777e-4

In [49]:
# Добавляю гетерогенность во времени
m2 = feols(log(v/v19) ~ year/log(deaths) + year/log(deaths_j) +
year/C8EV_International_travel_controls + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
m2 %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                       0.078145   0.077102  1.013525
year2022                                       0.132887   0.088895  1.494875
year2020:log(deaths)                          -0.014540   0.006968 -2.086681
year2021:log(deaths)                          -0.014148   0.006784 -2.085578
year2022:log(deaths)                          -0.021569   0.007546 -2.858254
year2020:log(deaths_j)                        -0.010634   0.006580 -1.616124
year2021:log(deaths_j)                         0.001444   0.006831  0.211366
year2022:log(deaths_j)                         0.003974   0.007462  0.532606
year2020:C8EV_International_travel_controls   -0.057193   0.025847 -2.212732
year2021:C8EV_International_travel_controls   -0.026873   0.012924 -2.079314
year2022:C8EV_International

In [50]:
etable(m1, m2, vcov = 'twoway')

,,m1,m2
,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19)
2,,,
3,log(deaths),-0.0156* (0.0061),
4,log(deaths_j),-0.0032 (0.0053),
5,C8EV_International_travel_controls,-0.0212 (0.0139),
6,C8EV_International_travel_controls_j,-0.0212 (0.0138),
7,year2021,,0.0781 (0.0771)
8,year2022,,0.1329 (0.0889)
9,year2020 x log(deaths),,-0.0145* (0.0070)


In [ ]:
# Норм ли, что выше были панельные регрессии, а эта пространственная? Как их тогда сравнивать?
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
m3 = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls + C8EV_International_travel_controls_j|
 log(deaths) + log(deaths_j) ~ 
 median_age + median_age_j + population_density + population_density_j,
 data = df %>% filter(year == '2020'))
m3 %>% summary(cluster = ~ i+j)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 3,419, IV: 3,381/450).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths), log(deaths_j)
                  Instr.   : median_age, median_age_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 18,391
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
(Intercept)                           0.026734   0.065750  0.406599 0.684842
fit_log(deaths)                       0.009457   0.006338  1.492044 0.137644
fit_log(deaths_j)                    -0.012174   0.007311 -1.665129 0.097832
C8EV_International_travel_controls   -0.014747   0.017973 -0.820538 0.413123
C8EV_International_travel_controls_j -0.034874   0.017615 -1.979781 0.049433
                                        
(Intercept)                             
fit_log(deaths)                         
fit_log(deaths_j)                    .  
C8EV_International_travel_controls      
C8EV_International_travel_controls_j *  
---
Sig